# Text Classification Model

## Load Data

In [ ]:
import pandas as pd
import pickle

In [ ]:
dataset = pd.read_csv('dataset_tweet_sentimen_tayangan_tv.csv')

In [ ]:
dataset

In [ ]:
dataset['Acara TV'].unique()

In [ ]:
dataset.groupby(['Acara TV','Sentiment'])['Id'].count().reset_index()

## Data Preparation & Pre-processing

In [ ]:
dataset.isnull().sum()

In [ ]:
import preprocessor as p

In [ ]:
clean_text = []
for text in dataset['Text Tweet']:
    clean_text.append(p.clean(text))

In [ ]:
lower_text=[]
for text in clean_text:
    lower_text.append(text.lower())

In [ ]:
import re
def remove_punct(text):
    clean_text = re.sub(r'[^\w\s]', '', text)
    return clean_text

In [ ]:
no_punct_text=[]
for text in lower_text:
    no_punct_text.append(remove_punct(text))

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
stop_factory = StopWordRemoverFactory()

In [ ]:
more_stopword = [
    'lu','gua','yg'
]
data = stop_factory.get_stop_words()+more_stopword
dictionary = ArrayDictionary(data)
stopwords = StopWordRemover(dictionary)

In [ ]:
pickle.dump(stopwords, open('stopwords.pkl', 'wb'))

In [ ]:
no_stopwords_text=[]
for text in no_punct_text:
    no_stopwords_text.append(stopwords.remove(text))

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
pickle.dump(stemmer, open('stemmer.pkl', 'wb'))

In [ ]:
stemmed_text = []
for text in no_stopwords_text:
    stemmed_text.append(stemmer.stem(text))

In [ ]:
stemmed_text

In [ ]:
dataset['cleaned_text'] = stemmed_text

In [ ]:
token=[]
for text in stemmed_text:
    token.extend(text.split())

In [ ]:
token

## Data Exploration

### All Document

In [ ]:
import itertools
all_words = list(itertools.chain(token))

In [ ]:
all_words

In [ ]:
import collections
counts_words = collections.Counter(all_words)

In [ ]:
counts_words

In [ ]:
df_word_freq = pd.DataFrame(counts_words.most_common(30),
                             columns=['words', 'count'])

df_word_freq

### Negative Document

In [ ]:
negative_doc = dataset[dataset['Sentiment']=="negative"]['cleaned_text']

In [ ]:
token_neg=[]
for text in negative_doc:
    token_neg.extend(text.split())

In [ ]:
all_words_neg = list(itertools.chain(token_neg))

In [ ]:
all_words_neg

In [ ]:
counts_words = collections.Counter(all_words_neg)

In [ ]:
df_word_freq = pd.DataFrame(counts_words.most_common(50),
                             columns=['words', 'count'])

df_word_freq

### Positive Document

In [ ]:
pos_doc = dataset[dataset['Sentiment']=="positive"]['cleaned_text']

In [ ]:
token_pos=[]
for text in pos_doc:
    token_pos.extend(text.split())

In [ ]:
all_words_pos = list(itertools.chain(token_pos))

In [ ]:
counts_words = collections.Counter(all_words_pos)

In [ ]:
df_word_freq = pd.DataFrame(counts_words.most_common(50),
                             columns=['words', 'count'])

df_word_freq

In [ ]:
dataset['label'] = list(map(lambda x: 1 if x=="positive" else 0,dataset['Sentiment']))

## Feature Transformation

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset['cleaned_text'], dataset['label'], random_state = 0,test_size=0.2)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

In [ ]:
X_train_counts = tfidf.fit_transform(X_train)
X_test_counts = tfidf.transform(X_test)

In [ ]:
pickle.dump(tfidf, open('feature_transformation.pkl', 'wb'))

In [ ]:
X_test_counts

In [ ]:
X_train_counts

## Modelling

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import BernoulliNB
BNBclf = BernoulliNB()
BNBclf.fit(X_train_counts, y_train)

In [ ]:
pickle.dump(BNBclf, open('naive_bayes_classifier_model.pkl', 'wb'))

In [ ]:
y_pred = BNBclf.predict(X_test_counts)

In [ ]:
y_pred

In [ ]:
import numpy as np
np.array(y_test)

In [ ]:
metric = {"model":[],
         "confusion_matrix":[],
         "auc":[],
         "accuracy":[]}

In [ ]:
metric['model'].append("NaiveBayes")

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

conf_ = confusion_matrix(y_test, y_pred)
metric["confusion_matrix"].append(conf_)
conf_

In [ ]:
_auc = roc_auc_score(y_test, y_pred)
metric["auc"].append(_auc)
_auc

In [ ]:
acc_ = accuracy_score(y_test, y_pred)
metric["accuracy"].append(acc_)
acc_

### Logistics Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train_counts, y_train)

In [ ]:
pickle.dump(clf, open('logistic_regression_classifier_model.pkl', 'wb'))

In [ ]:
y_pred = clf.predict(X_test_counts)

In [ ]:
metric['model'].append("LogisticRegression")

In [ ]:
conf_ = confusion_matrix(y_test, y_pred)
metric["confusion_matrix"].append(conf_)
conf_

In [ ]:
_auc = roc_auc_score(y_test, y_pred)
metric["auc"].append(_auc)
_auc

In [ ]:
acc_ = accuracy_score(y_test, y_pred)
metric["accuracy"].append(acc_)
acc_

### Support Vector Machine

In [ ]:
from sklearn import svm
svclf = svm.SVC()
svclf.fit(X_train_counts, y_train)

In [ ]:
pickle.dump(svclf, open('SVM_classifier_model.pkl', 'wb'))

In [ ]:
y_pred = svclf.predict(X_test_counts)

In [ ]:
metric['model'].append("SVM")

In [ ]:
conf_ = confusion_matrix(y_test, y_pred)
metric["confusion_matrix"].append(conf_)
conf_

In [ ]:
_auc = roc_auc_score(y_test, y_pred)
metric["auc"].append(_auc)
_auc

In [ ]:
acc_ = accuracy_score(y_test, y_pred)
metric["accuracy"].append(acc_)
acc_

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfclf = RandomForestClassifier()
rfclf.fit(X_train_counts, y_train)

In [ ]:
pickle.dump(rfclf, open('random_forest_classifier_model.pkl', 'wb'))

In [ ]:
y_pred = rfclf.predict(X_test_counts)

In [ ]:
metric['model'].append("Random Forest")

In [ ]:
conf_ = confusion_matrix(y_test, y_pred)
metric["confusion_matrix"].append(conf_)
conf_

In [ ]:
_auc = roc_auc_score(y_test, y_pred)
metric["auc"].append(_auc)
_auc

In [ ]:
acc_ = accuracy_score(y_test, y_pred)
metric["accuracy"].append(acc_)
acc_

In [ ]:
pickle.dump(metric, open('classifier_model_metric.pkl', 'wb'))